In [205]:
from function import *
from data_preparation import *
from evaluation import *

from openpyxl import Workbook
import pandas as pd
import numpy as np
import operator

In [15]:
import pandas as pd
import numpy as np
from rdkit.Chem import AllChem
from rdkit import Chem
import operator


def transform_data(input_file_name, output_file_name):
    from collections import OrderedDict

    f = open('../dataset/data_preprocessing/lifechem123_cleaned_2017_03_10.smi', 'r')
    mol_smile_dict = {}

    for line in f:
        line = line.strip()
        row = line.split(' ')
        smiles = row[0]
        mol = row[1]
        mol_smile_dict[mol] = smiles

    mol_smile_map = sorted(mol_smile_dict.items())
    
    molecules = [item[0] for item in mol_smile_map]
    smiles = [item[1] for item in mol_smile_map]
    
    df = pd.DataFrame({'Molecule': molecules, 'SMILES': smiles})
    result = pd.merge(df, Keck_Pria_Retest, on='Molecule', how='outer')
    result = pd.merge(result, Keck_Pria_FP, on='Molecule', how='outer')
    result = pd.merge(result, Keck_Pria_Primary, on='Molecule', how='outer')
    result = pd.merge(result, Keck_RMI, on='Molecule', how='outer')
    result = pd.merge(result, Keck_RMI_cdd, on='Molecule', how='outer')

    result.to_csv(output_file_name, index=None)

    return


if __name__ == '__main__':
    discrete_file = pd.ExcelFile('../dataset/data_preprocessing/screening_smsf_actives_2017_03_10.xlsx')
    Keck_Pria_Retest = discrete_file.parse('Keck_Pria_Retest')
    Keck_Pria_FP = discrete_file.parse('Keck_Pria_FP')
    Keck_RMI = discrete_file.parse('Keck_RMI')
    Xing_MTDH_Retest = discrete_file.parse('Xing_MTDH_Retest')
    Xing_MTDH_DR = discrete_file.parse('Xing_MTDH_DR')
    
    continuous_file = pd.ExcelFile('../dataset/data_preprocessing/screening_smsf_continuous_2017_03_10.xlsx')
    Keck_Pria_Primary = continuous_file.parse('Keck_Pria_Primary')
    Keck_RMI_cdd = continuous_file.parse('Keck_RMI_cdd')
    Xing_MTDH_cdd = continuous_file.parse('Xing_MTDH_cdd')
        
    transform_data('../dataset/data_preprocessing/lifechem123_cleaned_2017_03_10.smi', '../dataset/keck_complete.csv')

In [121]:
print Keck_Pria_Retest.dtypes
print
print Keck_Pria_FP.dtypes
print
print Keck_RMI.dtypes
print
print Xing_MTDH_Retest.dtypes
print
print Xing_MTDH_DR.dtypes

Molecule               object
Keck_Pria_AS_Retest     int64
dtype: object

Molecule             object
Keck_Pria_FP_data     int64
dtype: object

Molecule        object
Keck_RMI_cdd     int64
dtype: object

Molecule                 object
Xing_MTDH-SND1_retest     int64
dtype: object

Molecule             object
Xing_MTDH-SND1_DR     int64
dtype: object


In [122]:
print Keck_Pria_Primary.dtypes
print
print Keck_RMI_cdd.dtypes
print
print Xing_MTDH_cdd.dtypes

Molecule                 object
Keck_Pria_Continuous    float64
dtype: object

Molecule                  object
FP counts % inhibition     int64
dtype: object

Molecule                      object
MTDH-SND1: FRET_decrease     float64
MTDH-SND1: CFP_increase      float64
MTDH-SND1: RATIO_decrease    float64
dtype: object


In [258]:
from collections import OrderedDict

f = open('../dataset/data_preprocessing/lifechem123_cleaned_2017_03_10.smi', 'r')
mol_smile_dict = {}

for line in f:
    line = line.strip()
    row = line.split(' ')
    smiles = row[0]
    mol = row[1]
    mol_smile_dict[mol] = smiles
    
mol_smile_map = sorted(mol_smile_dict.items())

In [339]:
molecules = [item[0] for item in mol_smile_map]
smiles = [item[1] for item in mol_smile_map]

In [ ]:
df = pd.DataFrame({'Molecule': molecules, 'SMILES': smiles})
result = pd.merge(df, Keck_Pria_Retest, on='Molecule', how='outer')
result = pd.merge(result, Keck_Pria_FP, on='Molecule', how='outer')
result = pd.merge(result, Keck_Pria_Primary, on='Molecule', how='outer')
result = pd.merge(result, Keck_RMI, on='Molecule', how='outer')
result = pd.merge(result, Keck_RMI_cdd, on='Molecule', how='outer')

result

In [340]:
# molecule_keys = Keck_Pria_Retest['Molecule']

# target_keck_pria_retest = Keck_Pria_Retest['Keck_Pria_AS_Retest']
# target_keck_pria_fp = Keck_Pria_FP['Keck_Pria_FP_data']
# target_keck_rmi = Keck_RMI['Keck_RMI_cdd']
# target_xing_retest = Xing_MTDH_Retest['Xing_MTDH-SND1_retest']
# target_xing_dr = Xing_MTDH_DR['Xing_MTDH-SND1_DR']

# target_keck_pria_primary = Keck_Pria_Primary['Keck_Pria_Continuous']
# target_keck_rmi = Keck_RMI_cdd['FP counts % inhibition']

In [341]:
def transform(dataframe, label_index, mapping):
    complete_keys = [item[0] for item in mapping]
    neo_dict = {}
    for k in complete_keys:
        neo_dict[k] = np.NaN
    
    for ix, row in dataframe.iterrows():
        mol = row[0].strip()
        value = row[label_index]
        neo_dict[mol] = value
    
    neo_dict = sorted(neo_dict.items())
    neo_values = [item[1] for item in neo_dict]
    return neo_values

def count_one(arr):
    cnt = 0
    for x in arr:
        if x == 1:
            cnt += 1
    print cnt
    return

def count_smaller(arr, threshold=50):
    cnt = 0
    for x in arr:
        if x >= threshold:
            cnt += 1
    print cnt
    return

In [342]:
target_keck_pria_retest = transform(Keck_Pria_Retest, 'Keck_Pria_AS_Retest', mol_smile_map)
count_one(target_keck_pria_retest)

79


In [343]:
target_keck_pria_fp = transform(Keck_Pria_FP, 'Keck_Pria_FP_data', mol_smile_map)
count_one(target_keck_pria_fp)

24


In [344]:
target_keck_rmi_cdd = transform(Keck_RMI, 'Keck_RMI_cdd', mol_smile_map)
count_one(target_keck_rmi)

2134


In [345]:
target_xing_retest = transform(Xing_MTDH_Retest, 'Xing_MTDH-SND1_retest', mol_smile_map)
count_one(target_xing_retest)

435


In [346]:
target_xing_dr = transform(Xing_MTDH_DR, 'Xing_MTDH-SND1_DR', mol_smile_map)
count_one(target_xing_dr)

21


In [347]:
target_keck_pria_primary = transform(Keck_Pria_Primary, 'Keck_Pria_Continuous', mol_smile_map)
count_smaller(target_keck_pria_primary, threshold=80)

117


In [348]:
target_keck_rmi_inhibition = transform(Keck_RMI_cdd, 'FP counts % inhibition', mol_smile_map)
count_smaller(target_keck_rmi, threshold=80)

107


In [349]:
df = pd.DataFrame({'Molecule': molecule_keys,
                   'SMILES': values,
                   'Keck Pria Retest': target_keck_pria_retest,
                   'Keck Pria FP': target_keck_pria_fp,
                   'Keck Pria Cts': target_keck_pria_primary,
                   'Keck RMI cdd': target_keck_rmi_cdd,
                   'Keck RMI inhibition': target_keck_rmi_inhibition})

df = df[['Molecule',
         'SMILES',
         'Keck Pria Retest',
         'Keck Pria FP',
         'Keck Pria Cts',
         'Keck RMI cdd',
         'Keck RMI inhibition']]

In [350]:
file_ = '../dataset/temp.csv'
df.to_csv(file_, index=None)

temp = pd.read_csv(file_)

In [351]:
temp[temp['Keck RMI cdd']==1]

,Molecule,SMILES,Keck Pria Retest,Keck Pria FP,Keck Pria Cts,Keck RMI cdd,Keck RMI inhibition
145,SMSSF-0014846,c1cc(ccc1C(=O)Nc2nc3c(s2)CCCC3)OCCCC,0,0,8.826335,1.0,60.0
147,SMSSF-0014848,c1ccc(c(c1)c2csc(n2)NC(=O)C(C)C)OC,0,0,-2.023913,1.0,36.0
148,SMSSF-0014849,C(#N)c1ccc(cc1)C(=O)Nc2c(c3c(s2)CCCC3)C#N,0,0,-3.888881,1.0,19.0
214,SMSSF-0014915,c1ccc2c(c1)nc(s2)c3c4c(sc3NC(=O)c5ccc(cc5)OCC)...,0,0,-16.912787,1.0,107.0
258,SMSSF-0014959,c1ccc2c(c1)nc(s2)c3c4c(sc3NC(=O)c5c(cccc5F)F)C...,0,0,-16.037252,1.0,77.0
302,SMSSF-0015003,c1ccc2c(c1)nc(s2)c3c4c(sc3NC(=O)c5ccc(s5)Cl)CN...,0,0,-11.438717,1.0,90.0
362,SMSSF-0015063,c1ccc(cc1)N(C)S(=O)(=O)c2ccc(cc2)C(=O)Nc3c(c4c...,0,0,-20.471004,1.0,37.0
393,SMSSF-0015094,c1ccc2c(c1)-c3c(sc(n3)NC(=O)c4nc5ccccc5s4)C2,0,0,0.936168,1.0,151.0
945,SMSSF-0015646,c1(c(c([nH]c1C)C(=O)CN2CCCCCC2)C)C(=O)OC,0,0,-4.959869,1.0,61.0
1446,SMSSF-0016147,c1cc2c(cc1C(=O)OC)sc(n2)NC(=O)c3ccc(cc3)N(C)C,0,0,-5.298039,1.0,86.0


In [352]:
Keck_RMI[Keck_RMI['Keck_RMI_cdd']==1]

,Molecule,Keck_RMI_cdd
133,SMSSF-0014846,1
135,SMSSF-0014848,1
136,SMSSF-0014849,1
202,SMSSF-0014915,1
246,SMSSF-0014959,1
290,SMSSF-0015003,1
350,SMSSF-0015063,1
381,SMSSF-0015094,1
933,SMSSF-0015646,1
1434,SMSSF-0016147,1


In [414]:
df = pd.DataFrame({'Molecule': molecules, 'SMILES': smiles})
result = pd.merge(df, Keck_Pria_Retest, on='Molecule', how='outer')
result = pd.merge(result, Keck_Pria_FP, on='Molecule', how='outer')
result = pd.merge(result, Keck_Pria_Primary, on='Molecule', how='outer')
result = pd.merge(result, Keck_RMI, on='Molecule', how='outer')
result = pd.merge(result, Keck_RMI_cdd, on='Molecule', how='outer')

result

,Molecule,SMILES,Keck_Pria_AS_Retest,Keck_Pria_FP_data,Keck_Pria_Continuous,Keck_RMI_cdd,FP counts % inhibition
0,SMSSF-0000087,C(#N)c1c2c(sc1NC(=O)c3ccccc3F)CCCC2,0,0,1.400833,0.0,-20.0
1,SMSSF-0000321,C(#N)c1c2c(sc1NC(=O)CN3CCOCC3)CCCC2,0,0,-6.215696,0.0,-17.0
2,SMSSF-0000406,c1ccc(cc1)C2(C(=O)NC(=O)N2)c3ccccc3,0,0,-7.824709,0.0,-24.0
3,SMSSF-0001064,c1ccc(cc1)c2c3ccccc3[nH]c2c4ccccc4,0,0,-7.897257,NaN,NaN
4,SMSSF-0001273,c1ccc(cc1)C(C(=O)O)NC(=O)C,0,0,-8.528123,NaN,NaN
5,SMSSF-0001643,c1c(nn2c1[nH]c(cc2=O)COC)C,0,0,-2.863364,0.0,-21.0
6,SMSSF-0001907,c1nc2c(c(n1)SC)nc[nH]2,0,0,-4.671514,0.0,-22.0
7,SMSSF-0002135,c1ccc2c(c1)c(nc(n2)O)O,0,0,-4.266247,0.0,-28.0
8,SMSSF-0002799,c1c([nH]c(=S)nc1C)C,0,0,-4.398751,0.0,-21.0
9,SMSSF-0002932,c1ccc(cc1)C(=O)c2cc(ccc2NC(=O)CN3CCOCC3)Cl,0,0,-3.320914,0.0,-24.0


In [415]:
file_ = '../dataset/temp2.csv'
result.to_csv(file_, index=None)

In [416]:
result

,Molecule,SMILES,Keck_Pria_AS_Retest,Keck_Pria_FP_data,Keck_Pria_Continuous,Keck_RMI_cdd,FP counts % inhibition
0,SMSSF-0000087,C(#N)c1c2c(sc1NC(=O)c3ccccc3F)CCCC2,0,0,1.400833,0.0,-20.0
1,SMSSF-0000321,C(#N)c1c2c(sc1NC(=O)CN3CCOCC3)CCCC2,0,0,-6.215696,0.0,-17.0
2,SMSSF-0000406,c1ccc(cc1)C2(C(=O)NC(=O)N2)c3ccccc3,0,0,-7.824709,0.0,-24.0
3,SMSSF-0001064,c1ccc(cc1)c2c3ccccc3[nH]c2c4ccccc4,0,0,-7.897257,NaN,NaN
4,SMSSF-0001273,c1ccc(cc1)C(C(=O)O)NC(=O)C,0,0,-8.528123,NaN,NaN
5,SMSSF-0001643,c1c(nn2c1[nH]c(cc2=O)COC)C,0,0,-2.863364,0.0,-21.0
6,SMSSF-0001907,c1nc2c(c(n1)SC)nc[nH]2,0,0,-4.671514,0.0,-22.0
7,SMSSF-0002135,c1ccc2c(c1)c(nc(n2)O)O,0,0,-4.266247,0.0,-28.0
8,SMSSF-0002799,c1c([nH]c(=S)nc1C)C,0,0,-4.398751,0.0,-21.0
9,SMSSF-0002932,c1ccc(cc1)C(=O)c2cc(ccc2NC(=O)CN3CCOCC3)Cl,0,0,-3.320914,0.0,-24.0


In [420]:
def count_values(row):
    cnt = 0
    if row[2] == 1:
        cnt += 1
    if row[3] == 1:
        cnt += 1
    if row[5] == 1:
        cnt += 1
    return cnt


a = 0
b = 0
c = 0
count_dict = {}
for ix, row in result.iterrows():
    cnt = count_values(row)
    if cnt not in count_dict.keys():
        count_dict[cnt] = 0
    count_dict[cnt] = count_dict[cnt] + 1
    if row[2] == 1:
        a += 1
    if row[3] == 1:
        b += 1
    if row[5] == 1:
        c += 1
print a, b, c
print count_dict

79 24 230
{0: 72094, 1: 325, 2: 4}


In [404]:
a = 0
b = 0
c = 0
for ix, row in temp.iterrows():
    if row[2] == 1:
        a += 1
    if row[3] == 1:
        b += 1
    if row[5] == 1:
        c += 1
print a, b, c

79 24 230


# Transform data

In [26]:
discrete_file = pd.ExcelFile('../dataset/screening_smsf_actives_2017_03_10.xlsx')
updated_binary_label = discrete_file.parse('Keck_Pria_Retest')
transform_data('../dataset/lc123_keckdata.sdf', '../dataset/keck_updated_complete.csv', updated_binary_label)

84075 items in all, 62608 are good, 1736 are bad


# Split Unique Data and Complement Data

In [27]:
mol_whole = updated_binary_label['Molecule'].tolist()
print len(mol_whole)

temp = pd.read_csv('../dataset/keck_updated_complete.csv')
smaller = set(temp['molecule ID(RegID)'].tolist())
print len(smaller)

output_file = open('../dataset/complement_data.csv', 'w')
i = 0
for m in mol_whole:
    if m not in smaller:
        i += 1
        print >> output_file, m
print i

72423
62608
9815


In [28]:
print len(mol_whole)

72423


In [29]:
file_name_list = ['../dataset/LC1_CDD_SD_20160818.sdf',
                  '../dataset/LC2_CDD_SD_20160818.sdf',
                  '../dataset/LC3_CDD_SD_20160818.sdf']

whole_set = set()
for file_name in file_name_list:
    suppl = Chem.SDMolSupplier(file_name)
    unique = set()
    for mol in suppl:
        if mol is None:
            continue
        name = mol.GetProp("Molecule Name")
        unique.add(name)
#         whole_set.add(name)
    whole_set = whole_set | unique
    print(len(unique))
print len(whole_set)

24992
24850
24952
72466


In [30]:
print len(mol_whole)

72423


In [31]:
i = 0
for m in whole_set:
    if m not in mol_whole:
        i += 1
        print m
print i

SMSSF-0058605
SMSSF-0548341
SMSSF-0059064
SMSSF-0060689
SMSSF-0061157
SMSSF-0053410
SMSSF-0054699
SMSSF-0061407
SMSSF-0058779
SMSSF-0061613
SMSSF-0058823
SMSSF-0041248
SMSSF-0058518
SMSSF-0042146
SMSSF-0059972
SMSSF-0063027
SMSSF-0062983
SMSSF-0548201
SMSSF-0548381
SMSSF-0060733
SMSSF-0548369
SMSSF-0046450
SMSSF-0059947
SMSSF-0548255
SMSSF-0063888
SMSSF-0060022
SMSSF-0548358
SMSSF-0041136
SMSSF-0058735
SMSSF-0060882
SMSSF-0053366
SMSSF-0053364
SMSSF-0059682
SMSSF-0058649
SMSSF-0061113
SMSSF-0058561
SMSSF-0058693
SMSSF-0054655
SMSSF-0050446
SMSSF-0062870
SMSSF-0063836
SMSSF-0053408
SMSSF-0548326
SMSSF-0053539
SMSSF-0064392
SMSSF-0548238
SMSSF-0548216
SMSSF-0046494
SMSSF-0060388
SMSSF-0548229
SMSSF-0040590
51


# Split data into k-fold

In [4]:
k = 5
output_file_list = ['../dataset/{}_fold_split_data/fold_1_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_2_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_3_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_4_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_5_data.csv.gz'.format(k)]
split_data('../dataset/keck_updated_complete.csv', output_file_list, k)

In [5]:
k = 10
output_file_list = ['../dataset/{}_fold_split_data/fold_1_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_2_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_3_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_4_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_5_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_6_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_7_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_8_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_9_data.csv.gz'.format(k),
                    '../dataset/{}_fold_split_data/fold_10_data.csv.gz'.format(k)]
split_data('../dataset/keck_updated_complete.csv', output_file_list, k)

# Merge data from splitting folds to form training and testing

In [6]:
train_pd = read_merged_data(output_file_list[:9])
test_pd = read_merged_data([output_file_list[9]])

# Test feature- and label- extraction

In [7]:
print train_pd.dtypes

X_train, y_train = extract_feature_and_label(train_pd)
X_test, y_test = extract_feature_and_label(test_pd)

Unnamed: 0            object
molecule ID(RegID)    object
library               object
existing SMILES       object
generated SMILES      object
1024_fingerprint      object
Pria_SSB_%INH         object
true_label            object
dtype: object


# Test evaluation metrics

In [8]:
y_actual = np.eye(4)
y_pred = np.eye(4)
y_pred[2] = np.array([0, 0, 1, 1])
y_pred[3] = np.array([0, 0, 1, 0])

for i in range(4):
    print roc_auc_multi(y_actual, y_pred, [i], np.mean)
    print roc_auc_multi(y_actual, y_pred, [i], np.median)
    print enrichment_factor_multi(y_actual, y_pred, 0.5)
    print

1.0
1.0
[[1.0, 2.0], [1.0, 2.0], [1.0, 2.0], [1.0, 2.0]]

1.0
1.0
[[1.0, 2.0], [1.0, 2.0], [1.0, 2.0], [1.0, 2.0]]

0.833333333333
0.833333333333
[[1.0, 2.0], [1.0, 2.0], [1.0, 2.0], [1.0, 2.0]]

0.333333333333
0.333333333333
[[1.0, 2.0], [1.0, 2.0], [1.0, 2.0], [1.0, 2.0]]



# Get updated data, and give some analysis

In [9]:
discrete_file = pd.ExcelFile('../dataset/screening_smsf_actives.xlsx')
continuous_file = pd.ExcelFile('../dataset/screening_smsf_continuous.xlsx')

print discrete_file.sheet_names
print continuous_file.sheet_names

[u'Keck_Pria_Retest', u'Keck_Pria_FP', u'Keck_RMI', u'Xing_MTDH_Retest', u'Xing_MTDH_DR']
[u'Keck_Pria_Primary', u'Keck_RMI_cdd', u'Xing_MTDH_cdd']


In [10]:
binary_data = discrete_file.parse('Keck_Pria_Retest')
print 'reset binary labels:'
print binary_data.dtypes

print

continuous_data = continuous_file.parse('Keck_Pria_Primary')
print 'cts labels:'
print continuous_data.dtypes

reset binary labels:
Molecule               object
Keck_Pria_AS_Retest     int64
dtype: object

cts labels:
Molecule                 object
Keck_Pria_Continuous    float64
dtype: object


# List all active compounds (updated version)

In [11]:
print binary_data[binary_data['Keck_Pria_AS_Retest']>0]

            Molecule  Keck_Pria_AS_Retest
576    SMSSF-0015261                    1
7315   SMSSF-0021761                    1
13763  SMSSF-0027944                    1
16138  SMSSF-0030209                    1
16803  SMSSF-0030826                    1
16837  SMSSF-0030860                    1
16901  SMSSF-0030922                    1
16907  SMSSF-0030928                    1
19073  SMSSF-0032984                    1
25686  SMSSF-0039184                    1
26232  SMSSF-0039974                    1
26511  SMSSF-0040244                    1
29310  SMSSF-0042907                    1
29965  SMSSF-0043541                    1
30010  SMSSF-0043585                    1
30569  SMSSF-0044122                    1
30603  SMSSF-0044155                    1
30614  SMSSF-0044166                    1
30661  SMSSF-0044210                    1
30755  SMSSF-0044297                    1
30756  SMSSF-0044297                    1
30888  SMSSF-0044424                    1
31008  SMSSF-0044540              